In [9]:
import pandas as pd
from IPython.core.display import display

In [10]:
sleep = pd.read_csv('../data/SleepStudyData.csv')

Description of data

- Enough = Do you think that you get enough sleep?
- Hours = On average, how many hours of sleep do you get on a weeknight?
- PhoneReach = Do you sleep with your phone within arms reach?
- PhoneTime = Do you use your phone within 30 minutes of falling asleep?
- Tired = On a scale from 1 to 5, how tired are you throughout the day? (1 being not tired, 5 being very tired)
- Breakfast = Do you typically eat breakfast?


In [11]:
display(sleep.head())

,Enough,Hours,PhoneReach,PhoneTime,Tired,Breakfast
0,Yes,8.0,Yes,Yes,3,Yes
1,No,6.0,Yes,Yes,3,No
2,Yes,6.0,Yes,Yes,2,Yes
3,No,7.0,Yes,Yes,4,No
4,No,7.0,Yes,Yes,2,Yes


In [20]:
# check for missing values

print(sleep.shape)
print(sleep.isnull().sum())
print('\n')

# check whether any missing or error values
for col in sleep.columns.tolist():
    print(col, sleep[col].unique())

(104, 6)
Enough        0
Hours         2
PhoneReach    0
PhoneTime     0
Tired         0
Breakfast     0
dtype: int64


Enough ['Yes' 'No']
Hours [ 8.  6.  7.  4. 10.  9.  2.  5. nan]
PhoneReach ['Yes' 'No']
PhoneTime ['Yes' 'No']
Tired [3 2 4 5 1]
Breakfast ['Yes' 'No']


In [47]:
# check where missing values in sleep

missing_hours = sleep.loc[sleep['Hours'].isnull()]
print(sleep.isnull().sum() / len(sleep))

# decide to drop this missing values as its < 2% of total records
sleep.drop(missing_hours.index, inplace=True)
sleep.isnull().sum()

Enough        0.000000
Hours         0.019231
PhoneReach    0.000000
PhoneTime     0.000000
Tired         0.000000
Breakfast     0.000000
dtype: float64


Enough        0
Hours         0
PhoneReach    0
PhoneTime     0
Tired         0
Breakfast     0
dtype: int64